In [22]:
import warnings

import pandas as pd

from MLWrappers import SklearnBlackBox, PyTorchBlackBox, KerasBlackBox
from PrivacyAttacks import MiaPrivacyAttack, LabelOnlyPrivacyAttack, AloaPrivacyAttack
from MLPrivacyEvaluator import PrivacyEvaluator

warnings.simplefilter("ignore", UserWarning)

In [23]:
DS_NAME = 'adult'
DATA_FOLDER = f'./data/{DS_NAME}'

# we load the target black box model using our wrapper
# target = SklearnBlackBox(f'./models/{DS_NAME}_dt.pkl')
target = SklearnBlackBox(f'./models/{DS_NAME}_rf.pkl')
# target = PyTorchBlackBox(f'./models/{DS_NAME}_nn_torch.pt')
# target = KerasBlackBox(f'./models/{DS_NAME}_nn_keras.keras')

# We load the data used to train, test of the model, as well as the shadow data
train_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_set.csv', skipinitialspace=True)
test_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_test_set.csv', skipinitialspace=True)
shadow_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_shadow_set.csv', skipinitialspace=True)

In [24]:
shadow_tree_params = {'min_samples_leaf': 100,
                      'max_depth': 10}
shadow_forest_params = {'n_estimators': 100}

In [25]:
# We initialise the attacks, with the desired parameters for each
mia = MiaPrivacyAttack(target,
                       n_shadow_models=4)
label_only = LabelOnlyPrivacyAttack(target,
                                    n_shadow_models=1,
                                    shadow_model_type='rf',
                                    shadow_model_params=shadow_forest_params,
                                    n_noise_samples_fit=1000)
aloa = AloaPrivacyAttack(target,
                         n_shadow_models=1,
                         shadow_model_type='rf',
                         shadow_model_params=shadow_forest_params,
                         n_noise_samples_fit=1000,
                         shadow_test_size=0.5,
                         undersample_attack_dataset=True)
attacks = [mia, label_only, aloa]

In [26]:
# We initialise the PrivacyEvaluator object
# We pass the target model and the attacks we want to use
evaluator = PrivacyEvaluator(target, attacks)

In [27]:
# We use the fit() method to execute the attacks, starting from the shadow data
evaluator.fit(shadow_set)

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Each shadow model uses 100.000 % of the data


 64%|██████▍   | 3847/6034 [00:15<00:08, 255.16it/s]


KeyboardInterrupt: 

In [ ]:
# Then we can obtain the performances using the report() method
results = evaluator.report(train_set, test_set)
print(results)


100%|██████████| 32000/32000 [01:43<00:00, 310.01it/s]


{'mia_attack': {'classification_report': {'IN': {'precision': 0.8002239125423952, 'recall': 0.949296875, 'f1-score': 0.8684092980042524, 'support': 25600}, 'OUT': {'precision': 0.2041692213366033, 'recall': 0.05203125, 'f1-score': 0.08292865147553233, 'support': 6400}, 'accuracy': 0.76984375, 'macro avg': {'precision': 0.5021965669394992, 'recall': 0.5006640625000001, 'f1-score': 0.47566897473989234, 'support': 32000}, 'weighted avg': {'precision': 0.6810129743012368, 'recall': 0.76984375, 'f1-score': 0.7113131686985085, 'support': 32000}}}, 'label_only_attack': {'classification_report': {'IN': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 25600}, 'OUT': {'precision': 0.2, 'recall': 1.0, 'f1-score': 0.33333333333333337, 'support': 6400}, 'accuracy': 0.2, 'macro avg': {'precision': 0.1, 'recall': 0.5, 'f1-score': 0.16666666666666669, 'support': 32000}, 'weighted avg': {'precision': 0.04, 'recall': 0.2, 'f1-score': 0.06666666666666667, 'support': 32000}}}, 'aloa_attack': 

In [ ]:
results['mia_attack']['classification_report']

{'IN': {'precision': 0.8002239125423952,
  'recall': 0.949296875,
  'f1-score': 0.8684092980042524,
  'support': 25600},
 'OUT': {'precision': 0.2041692213366033,
  'recall': 0.05203125,
  'f1-score': 0.08292865147553233,
  'support': 6400},
 'accuracy': 0.76984375,
 'macro avg': {'precision': 0.5021965669394992,
  'recall': 0.5006640625000001,
  'f1-score': 0.47566897473989234,
  'support': 32000},
 'weighted avg': {'precision': 0.6810129743012368,
  'recall': 0.76984375,
  'f1-score': 0.7113131686985085,
  'support': 32000}}

In [ ]:
results['label_only_attack']['classification_report']

{'IN': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 25600},
 'OUT': {'precision': 0.2,
  'recall': 1.0,
  'f1-score': 0.33333333333333337,
  'support': 6400},
 'accuracy': 0.2,
 'macro avg': {'precision': 0.1,
  'recall': 0.5,
  'f1-score': 0.16666666666666669,
  'support': 32000},
 'weighted avg': {'precision': 0.04,
  'recall': 0.2,
  'f1-score': 0.06666666666666667,
  'support': 32000}}

In [ ]:
results['aloa_attack']['classification_report']

{'IN': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 25600},
 'OUT': {'precision': 0.2,
  'recall': 1.0,
  'f1-score': 0.33333333333333337,
  'support': 6400},
 'accuracy': 0.2,
 'macro avg': {'precision': 0.1,
  'recall': 0.5,
  'f1-score': 0.16666666666666669,
  'support': 32000},
 'weighted avg': {'precision': 0.04,
  'recall': 0.2,
  'f1-score': 0.06666666666666667,
  'support': 32000}}

In [ ]:
aloa.attack_model.threshold

0.9